# Goal: Classify Genetic Variations using Text from Clinical Evidence
A team at Memorial Sloan Kettering Cancer Center (MSKCC) spends counless hours every year manually reviewing evidence to classify genetic mutations. Once the most important variants have been identified new therapies can be developed. Having a machine learning algorithm to automatically classify genetic variations would allow the development of therapies much sooner. This competition is a challenge to develop classification models which analyze abstracts of medical articles and, based on their content accurately determine mutation effect (9 classes) of the genes discussed in them. For more information, please refer to the Kaggle competition: https://www.kaggle.com/c/msk-redefining-cancer-treatment#description. 

Here is an article that also gives context to a project such as this and the impact could have: https://www.forbes.com/sites/matthewherper/2017/06/03/a-new-cancer-drug-helped-almost-everyone-who-took-it-almost-heres-what-it-teaches-us/#12aad9136b25. 

This task is complex and hard to elvaluate the cross val score due to the data only representing some of the genes in the dataset. In the end, if my score is not great, I hope to have been able to better understand the use of machine learning in genomic analysis and explore its limitations. 


Also, there was a 2nd phase to this competition due to the labels for the test data being released by a third party before the competition closed. I may want to combine the training data from both phases to train the model if I am not able to get a good score from just the initial data set. 

#### Target: 9 classes that represent different levels of whether or not the variant is a driver or passenger mutation.
These are the class labels from OncoKB, but using this text in the competition is prohibited. 
1. Likely Loss-of-function
2. Likely Gain-of-function
3. Neutral
4. Loss-of-function
5. Likely Neutral
6. Inconclusive
7. Gain-of-function
8. Likely Switch-of-function
9. Switch-of-function

Using COSMIC and Genotype-Phenotype associations are not allowed due to the point of this project being modeling text.

You are not allowed to use conservation scores, or predictors of deleterious or tolerated mutations (such as Polyphen) because these tools also try to predict the effect of a mutation (which is very close to the goal of this competition) but they don't use text to do it

##### Outside resources that have been approved by kaggle for use in this competition but not necessary:
 -  Gene–disease associations: http://ctdbase.org/
 -  Text data from:
    - Encyclopedia_of_Molecular_Cell_Biology_and_Molecular_Medicine_16_volumes_Wiley_2006_
    - Cell-Molecular-Biology-Concepts-Experiments 7th edition
 - MESH topics, as well as extended information about those articles
 - Pre-trained Bio-NER taggers (May need to map the variations to RSIDs and search related articles in PubMed or PMC):
    - tmVar https://www.ncbi.nlm.nih.gov/research/bionlp/Tools/tmvar/
    - TaggerOne https://www.ncbi.nlm.nih.gov/research/bionlp/Tools/taggerone/
    - GNormPlus https://www.ncbi.nlm.nih.gov/research/bionlp/Tools/gnormplus/
 - Pathway and protein information of each gene (e.g. GO).
 - Pubmed dictionaries of genes names.
 - Mygene api (http://docs.mygene.info/en/v3/doc/data.html) It contains a lot of information about genes (such as family, overall description, etc). 
 - Gene expression from The Human Protein Atlas project ( https://www.ebi.ac.uk/gxa/experiments/E-PROT-3/Downloads )
 - The aminoacid sequences for the genes in the dataset, in FASTA.
 - Other papers from PubMed in addition to the training_text in order to pre-train NLP model
 - Genia, an annotated treebank for parsing biomed papers(http://www.geniaproject.org/)
 - You can use variations themselves as features. For example a 'Deletion' in a gene may be more likely to be 1. Similar case could be made for amino acid changes. Some amino acid changes are more deleterious than others.
 - Google's trained Word2Vec models and pretrained models from bio.nlplab.org
 - Sequence data, mutation positition data and protein structure data from Uniport
 - allele frequency information from gnomAD or ExAC
 - controlled vocabularies to guide Named Entity Recognition modles. Examples: such as ICD, SNOMED CT,NCI Thesaurus, CPT, MedDRA, SNOMED CT
 
 Note from Kaggle: "We knew the task was very difficult, so Iker from MSK suggested that we allow external data, in case anyone wants to build a knowledge base for gene mutations."

---
# Imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, \
HashingVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier

from nltk.corpus import stopwords

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

%matplotlib inline

In [4]:
train_var = pd.read_csv('Data/training_variants.zip')
train_txt = pd.read_csv('Data/training_text.zip', sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])
test_var = pd.read_csv('Data/test_variants.zip')
test_txt = pd.read_csv('Data/test_text.zip', sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])

---
# EDA

---
### Training Variants

In [ ]:
print(train_var.shape)
print(train_var.dtypes)
print(train_var.isnull().sum())
train_var.head()

In [ ]:
pd.DataFrame(train_var['Gene'].value_counts())

In [ ]:
pd.DataFrame(train_var['Variation'].value_counts())

In [ ]:
train_var['Class'].value_counts()

In [ ]:
plt.figure()
sns.countplot(x="Class", data=train_var)
plt.ylabel('Frequency')
plt.xlabel('Class')
plt.title("Distribution of genetic mutation classes")
plt.show()

##### Looking at the genes with 50 or more occurences in the training data set

In [ ]:
# BRCA1
BRCA1 = train_var.loc[train_var['Gene']=='BRCA1']
print(BRCA1['Class'].value_counts())

In [ ]:
# TP53
TP53 = train_var.loc[train_var['Gene']=='TP53']
print(TP53['Class'].value_counts())

In [ ]:
# EGFR
EGFR = train_var.loc[train_var['Gene']=='EGFR']
print(EGFR['Class'].value_counts())

In [ ]:
# PTEN
PTEN = train_var.loc[train_var['Gene']=='PTEN']
print(PTEN['Class'].value_counts())

In [ ]:
# BRCA2
BRCA2 = train_var.loc[train_var['Gene']=='BRCA2']
print(BRCA2['Class'].value_counts())

In [ ]:
# KIT
KIT = train_var.loc[train_var['Gene']=='KIT']
print(KIT['Class'].value_counts())

In [ ]:
# BRAF
BRAF = train_var.loc[train_var['Gene']=='BRAF']
print(BRAF['Class'].value_counts())

In [ ]:
# ERBB2
ERBB2 = train_var.loc[train_var['Gene']=='ERBB2']
print(ERBB2['Class'].value_counts())

In [ ]:
# ALK
ALK = train_var.loc[train_var['Gene']=='ALK']
print(ALK['Class'].value_counts())

In [ ]:
# PDGFRA
PDGFRA = train_var.loc[train_var['Gene']=='PDGFRA']
print(PDGFRA['Class'].value_counts())

In [ ]:
# PIK3CA
PIK3CA = train_var.loc[train_var['Gene']=='PIK3CA']
print(PIK3CA['Class'].value_counts())

In [ ]:
# CDKN2A
CDKN2A = train_var.loc[train_var['Gene']=='CDKN2A']
print(CDKN2A['Class'].value_counts())

In [ ]:
# FGFR2
FGFR2 = train_var.loc[train_var['Gene']=='FGFR2']
print(FGFR2['Class'].value_counts())

---
### Training Text

In [ ]:
print(train_txt.shape)
print(train_txt.isnull().sum())
train_txt.head()

In [ ]:
# len(train_txt['Test'])

---
### Test Variants

In [ ]:
print(test_var.shape)
test_var.head()

In [ ]:
print(test_txt.shape)
test_txt.head()

In [ ]:
print(train_txt.dtypes)
print(train_txt.isnull().sum())

# Feature Selection


Joining variant and text training dataframes and then splitting into X and y. Will need to get dummies to create a sparse matrix for the Gene and Variation columns and a multiclass target. 

In [5]:
df = train_var.merge(train_txt ,on='ID')
X = df[['Gene', 'Variation', 'Text']]
# X = pd.get_dummies(X, prefix=['gene', 'var'], prefix_sep='', columns=['Gene', 'Variation'])
print(X.shape)
X.head()

(3321, 3)


,Gene,Variation,Text
0,FAM58A,Truncating Mutations,Cyclin-dependent kinases (CDKs) regulate a var...
1,CBL,W802*,Abstract Background Non-small cell lung canc...
2,CBL,Q249E,Abstract Background Non-small cell lung canc...
3,CBL,N454D,Recent evidence has demonstrated that acquired...
4,CBL,L399V,Oncogenic mutations in the monomeric Casitas B...


In [6]:
y = df['Class']
print(y.shape)
# y = pd.get_dummies(y, prefix='class', prefix_sep='', columns=['Class'])
y.head()

(3321,)


0    1
1    2
2    2
3    3
4    4
Name: Class, dtype: int64

Train test split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.90, random_state=42)

In [8]:
print(X_train.shape, 
      X_test.shape, 
      y_train.shape, 
      y_test.shape
)

(332, 3) (2989, 3) (332,) (2989,)


# Demonstration of sweet sucksess
Using a not pipe

In [13]:
rf = RandomForestClassifier()

rf.fit(
    CountVectorizer().fit_transform(X_train['Text']).toarray(),
    y_train
)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [14]:
cross_val_score(rf, CountVectorizer().fit_transform(X_train['Text']).toarray(),
                y_train)

/home/greg/xtol/miniconda3/envs/ga/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


array([ 0.45535714,  0.44144144,  0.50458716])

# Demonstration of sweet failyour
<img src="http://nj1015.com/files/2012/01/Picture-15.png?w=600&h=0&zc=1&s=0&a=t&q=89" height="200" width="200" align="left">
Pipes ruin lives.

In [20]:
def cvItGood(cv_array):
    return CountVectorizer().fit_transform(cv_array).toarray()

In [18]:
# shape is correct after transform and densify!
CountVectorizer().fit_transform(X_train['Text']).toarray().shape

(332, 52441)

In [19]:
# and it's totally a numpy array!
type(CountVectorizer().fit_transform(X_train['Text']).toarray())

numpy.ndarray

In [21]:
pipe = Pipeline([
    ('cvtf', FunctionTransformer(cvItGood, validate=False)),
    ('clf', RandomForestClassifier())
])

Fit the best model params to the training data

In [22]:
pipe.fit(X_train['Text'], y_train)

Pipeline(steps=[('cvtf', FunctionTransformer(accept_sparse=False,
          func=<function cvItGood at 0x7f33a113d7b8>, inv_kw_args=None,
          inverse_func=None, kw_args=None, pass_y=False, validate=False)), ('clf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            ...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])

In [24]:
pipe.score(X_train['Text'], y_train)

0.95783132530120485

In [31]:
# still have some feature mismatch between test/train we'll need to fix
# but it's gettin thur
pipe.score(X_test.iloc[:100,:]['Text'], y_test.iloc[:100])

ValueError: Number of features of the model must match the input. Model n_features is 52441 and input n_features is 30523 